<a href="https://colab.research.google.com/github/sarasti2/AI_UdeA_2024-1/blob/main/fase-2/generate-data-and-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Configuraciones iniciales del entorno de trabajo

In [25]:
# Cargar las librerias que se usaran
import numpy as np # Algebra lineal
import pandas as pd # Dataframe tabulares
import os # Permite utilizar funcionalidades del sistema operativo
import seaborn as sns # Graficas estadisticas
from sklearn.model_selection import train_test_split # Creación de subconjuntos
from sklearn.preprocessing import LabelEncoder # Codificar etiquetas
import xgboost as xgb # Algoritmo para la generación de modelos xgboost
import matplotlib.pyplot as plt # Graficas
from matplotlib.pyplot import figure

In [26]:
# Cargar los datos desde Kaggle
# ¡IMPORTANTE! subir el archivo .json, que representa el "Token" de Kaggle que
# se genera según las instrucciones en el readme
os.environ['KAGGLE_CONFIG_DIR']="."
!kaggle competitions download -c predicting-red-hat-business-value
!unzip predicting-red-hat-business-value.zip

predicting-red-hat-business-value.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  predicting-red-hat-business-value.zip
replace act_test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 0. Cargar bases de datos

In [34]:
# La base de datos esta compuesta por una de personas, y otra de las actividades
# que hacen las personas, cada persona tiene un identificador unico que permite
# crea una sola base con ambas informaciones.

train_df = pd.read_csv('act_train.csv.zip')
people_df = pd.read_csv('people.csv.zip')


# 2. Crear una base de datos consolidada

In [35]:
# Por la cantidad de valores vacios, se procedera a eliminar los char_#

train_df = train_df.drop(['char_1', 'char_2', 'char_3',
                          'char_4', 'char_5', 'char_6',
                         'char_7', 'char_8', 'char_9',
                         'char_10'], axis=1)

# Se une por el identificador unico de persona
train_df.set_index('people_id')
people_df.set_index('people_id')
df = pd.merge(train_df, people_df)

# Dar el formato adecuado a las columnas
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['isweekend'] = (df['date'].dt.weekday >= 5).astype(int)
df = df.drop('date', axis = 1)

# 3. Preprocesamiento

In [36]:
# Dividir las columnas en categoricas y numericas
categorical_features = []
numeric_features = []
features = df.columns.values.tolist()
for col in features:
    if df[col].dtype != 'object':
        numeric_features.append(col)
    else:
        categorical_features.append(col)

# Codificar las columnas categoricas para que sean interpretables por el algoritmo

for col in categorical_features:
    le = LabelEncoder()
    le.fit(list(df[col].astype(str).values))
    df[col] = le.transform(list(df[col].astype(str).values))

# Creación de los sub-conjuntos de prueba y verificación
y = df['outcome']
X = df.drop('outcome', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 4. Modelo



In [37]:
# Se usara un xgboost debido a que es un problema de clasificación con una alta
# cantidad de datos

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=10, learning_rate=0.05, use_label_encoder=False)
gbm = gbm.fit(X_train, y_train)

# Guardar modelo
with open("model.pkl", "wb") as file:
    pickle.dump(gbm, file)

In [40]:
# Abrir el modelo guardado para verificar su funcionamiento
with open('model.pkl', 'rb') as file:
    _gbm = pickle.load(file)

# Verificación del rendimiento del modelo, en este caso se usara el criterio
# de exactitud
print("Exactitud conjunto de entrenamiento: {:.3f}".format(_gbm.score(X_train, y_train)))
print("Exactitud conjunto de verificacion: {:.3f}".format(_gbm.score(X_test, y_test)))

Exactitud conjunto de entrenamiento: 0.856
Exactitud conjunto de verificacion: 0.854
